# Movielens 영화 추천 

In [1]:
# 필요한 라이브러리 import
import numpy as np
import scipy
import implicit
import pandas as pd
from implicit.als import AlternatingLeastSquares

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [2]:
# 유저 데이터 불러오기
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 퍙점 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 유저 데이터와 영화 제목 데이터를 합병
movies_ratings = pd.merge(ratings, movies, on='movie_id')
movies_ratings

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [8]:
# 첫 번째 유저가 어떤 영화를 봤는지 확인
condition = (movies_ratings['user_id']== movies_ratings.loc[0, 'user_id'])
movies_ratings.loc[condition]

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1680,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2123,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
2734,1,3408,4,978300275,Erin Brockovich (2000),Drama
3957,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
5556,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
7808,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama
8474,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama
9764,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
10471,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [9]:
# 유저 수
movies_ratings['user_id'].nunique()

6039

In [10]:
# 영화 수
movies_ratings['movie_id'].nunique()

3628

In [11]:
# 인기 많은 영화
movies_count = movies_ratings.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 데이터 추가를 위해 영화타임의 최댓값 확인
movies_ratings['timestamp'].max()

1046454590

In [13]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 제목은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Men in Black (1997)' , 'Forrest Gump (1994)' ,'Toy Story (1995)' ,'Jurassic Park (1993)' ,'Titanic (1997)']
my_favorite_movie_id = ['1580', '356', '1', '480', '1721']
my_favorite_genre = ['Action|Adventure|Comedy|Sci-Fi', 'Comedy|Romance|War', "Animation|Children's|Comedy", 'Action|Adventure|Sci-Fi', 'Drama|Romance']

# 좋아하는 영화에 대한 데이터 프레임 생성
# 평점과 영화를 본 시간은 좋아하는 영화임으로 최대치로 줌
my_playlist = pd.DataFrame({'user_id': ['minki']*5, 'title': my_favorite, 'movie_id': my_favorite_movie_id, 'timestamp':[1046454590]*5, 'counts': [5.0]*5, 'genre': my_favorite_genre})

# movies_ratings 데이터 프레임에 좋아하는 영화 데이터 프레임 추가
if not movies_ratings.isin({'user_id':['minki']})['user_id'].any():  # user_id에 'minki'이라는 데이터가 없다면
    movies_ratings = movies_ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movies_ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp,title,genre
836473,5851,3607,5.0,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4.0,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3.0,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4.0,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5.0,1016563709,Identification of a Woman (Identificazione di ...,Drama
0,minki,1580,5.0,1046454590,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
1,minki,356,5.0,1046454590,Forrest Gump (1994),Comedy|Romance|War
2,minki,1,5.0,1046454590,Toy Story (1995),Animation|Children's|Comedy
3,minki,480,5.0,1046454590,Jurassic Park (1993),Action|Adventure|Sci-Fi
4,minki,1721,5.0,1046454590,Titanic (1997),Drama|Romance


In [14]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = movies_ratings['user_id'].unique()
title_unique = movies_ratings['title'].unique()
genre_unique = movies_ratings['genre'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}
genre_to_idx = {v:k for k,v in enumerate(genre_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['minki'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['Men in Black (1997)'])
print(genre_to_idx['Action|Adventure|Comedy|Sci-Fi'])

6039
175
75


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movies_ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movies_ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies_ratings['user_id'] = temp_user_data   # movies_ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = movies_ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movies_ratings):
    print('title column indexing OK!!')
    movies_ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')
    
temp_genre_data = movies_ratings['genre'].map(genre_to_idx.get).dropna()
if len(temp_genre_data) == len(movies_ratings):
    print('genre column indexing OK!!')
    movies_ratings['genre'] = temp_genre_data
else:
    print('genre column indexing Fail!!') 

movies_ratings

user_id column indexing OK!!
title column indexing OK!!
genre column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,1193,5.0,978300760,0,0
1,1,1193,5.0,978298413,0,0
2,2,1193,4.0,978220179,0,0
3,3,1193,4.0,978199279,0,0
4,4,1193,5.0,978158471,0,0
...,...,...,...,...,...,...
0,6039,1580,5.0,1046454590,175,75
1,6039,356,5.0,1046454590,160,69
2,6039,1,5.0,1046454590,40,3
3,6039,480,5.0,1046454590,107,52


In [17]:
# CSR matrix
from scipy.sparse import csr_matrix

num_user = movies_ratings['user_id'].nunique()
num_title = movies_ratings['title'].nunique()

csr_movies_ratings = csr_matrix((movies_ratings.counts, (movies_ratings.user_id, movies_ratings.title)), shape= (num_user, num_title))
csr_movies_ratings

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_movies_ratings_transpose = csr_movies_ratings.T
csr_movies_ratings_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_movies_ratings_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [21]:
# 내가 선호하는 영화에 대한 모델이 예측한 나의 선호도
minki, Titanic = user_to_idx['minki'], title_to_idx['Titanic (1997)']
minki_vector, Titanic_vector = als_model.user_factors[minki], als_model.item_factors[Titanic]

print('슝=3')

슝=3


In [22]:
minki_vector

array([ 0.84054244,  0.00196984, -0.42653146, -0.15589356,  0.645996  ,
       -0.16499314, -0.6079522 , -0.11688709,  0.15930124, -0.79022574,
        0.1654739 , -0.01021648, -0.09260057, -1.1672165 , -0.17992234,
       -0.06444978,  0.35414726,  0.7098924 ,  0.5102803 ,  0.34505367,
        0.32053933, -0.24353412,  0.07792471, -0.40633824, -0.02008162,
       -0.12239581, -0.31847304, -0.47422296,  0.48439214,  0.12336157,
        0.3177505 , -0.9037151 , -0.4387975 , -0.57887405,  0.76619464,
       -0.12581202,  0.32570672, -0.18653814,  0.32693073, -0.31852078,
       -0.98970354,  0.40977386, -0.19386944, -0.11994619,  0.44541693,
       -0.4105391 ,  0.70692694,  0.4372584 ,  0.08368804, -0.0682173 ,
        0.15148896, -0.32856798,  0.30232784, -0.183604  ,  1.2607722 ,
       -0.44918573, -1.2463961 ,  0.06791706,  0.14695202,  0.26198015,
        0.539811  , -0.5901391 , -0.5513191 , -0.9516208 ,  0.03942779,
       -0.13418281, -0.9253897 ,  0.60668755, -0.6815259 ,  1.16

In [23]:
Titanic_vector

array([-0.00322732, -0.00128015, -0.00891205,  0.02712655,  0.01806442,
        0.02103037,  0.00410982,  0.00380903,  0.02327291, -0.03156209,
       -0.03509643, -0.00120641, -0.03293729, -0.0254002 , -0.03497734,
        0.00245053, -0.00408059,  0.05363548,  0.01558309,  0.05036492,
        0.01696417,  0.00457572,  0.02691196, -0.01401375,  0.0341438 ,
       -0.00943946,  0.01477335, -0.01181838, -0.01464355,  0.01465671,
        0.00903289, -0.01049487,  0.02634182, -0.00169256,  0.01522245,
       -0.0201042 ,  0.02708454, -0.00231266,  0.01795178, -0.00538404,
        0.01418212,  0.03491827, -0.02642993, -0.00809844,  0.0274773 ,
       -0.01658611,  0.04828802, -0.01326014,  0.02198431, -0.01270995,
        0.00628138,  0.00556415,  0.03516948,  0.00967126,  0.0363859 ,
        0.00253263, -0.0302861 ,  0.00302227, -0.01965098,  0.00455292,
        0.03071934, -0.02411356, -0.01363382, -0.01692946,  0.01058077,
        0.0233086 , -0.02407791,  0.01482288,  0.02022628, -0.01

In [24]:
# 나의 Titanic에 대한 선호도
np.dot(minki_vector, Titanic_vector)

0.5811807

In [25]:
# 나의 E.T.에 대한 선호도
ET = title_to_idx['E.T. the Extra-Terrestrial (1982)']
ET_vector = als_model.item_factors[ET]
np.dot(minki_vector, ET_vector)

0.056161396

### 비슷한 영화 찾기

In [26]:
# Men in Black (1997)
favorite_title = 'Men in Black (1997)'
title = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title, N=15)
similar_title

[(175, 1.0),
 (107, 0.8559934),
 (92, 0.6333183),
 (62, 0.5942207),
 (150, 0.54666483),
 (124, 0.49795136),
 (82, 0.460491),
 (145, 0.45336866),
 (375, 0.44874397),
 (3466, 0.41347662),
 (138, 0.40194064),
 (179, 0.3766734),
 (544, 0.37020442),
 (542, 0.36233276),
 (2748, 0.35067904)]

In [27]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Matrix, The (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'True Lies (1994)',
 'Rock, The (1996)',
 'Stargate (1994)',
 'Rocky Horror Picture Show, The (1975)',
 'Bewegte Mann, Der (1994)']

In [28]:
# 위의 과정을 함수로 저장
def get_similar_title(title_name: str):
    title = title_to_idx[title_name]
    similar_title = als_model.similar_items(title)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [29]:
# Tarzan (1999)
get_similar_title('Tarzan (1999)')

['Tarzan (1999)',
 'Mulan (1998)',
 'Hunchback of Notre Dame, The (1996)',
 'Hercules (1997)',
 'Anastasia (1997)',
 'Pocahontas (1995)',
 'Balto (1995)',
 'Lion King, The (1994)',
 'Rescuers Down Under, The (1990)',
 'Antz (1998)']

### 유저에게 영화 추천하기

In [30]:
user = user_to_idx['minki']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_movies_ratings, N=20, filter_already_liked_items=True)
title_recommended

[(50, 0.4140494),
 (92, 0.40915257),
 (87, 0.34709936),
 (110, 0.34257144),
 (150, 0.3166988),
 (124, 0.30618912),
 (60, 0.29382613),
 (322, 0.27804264),
 (154, 0.26856533),
 (48, 0.25979078),
 (22, 0.25371313),
 (64, 0.25214496),
 (62, 0.24960685),
 (33, 0.24362391),
 (330, 0.2405965),
 (99, 0.24050951),
 (116, 0.24009827),
 (38, 0.23518185),
 (15, 0.23065555),
 (20, 0.220768)]

In [31]:
[idx_to_title[i[0]] for i in title_recommended]

['Toy Story 2 (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Braveheart (1995)',
 'Groundhog Day (1993)',
 'Independence Day (ID4) (1996)',
 'Matrix, The (1999)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Babe (1995)',
 'As Good As It Gets (1997)',
 'Saving Private Ryan (1998)',
 'Back to the Future (1985)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Total Recall (1990)',
 'Aladdin (1992)',
 'Lion King, The (1994)',
 'American Beauty (1999)',
 'Dances with Wolves (1990)',
 'Sixth Sense, The (1999)',
 'Airplane! (1980)',
 'Pleasantville (1998)']

In [32]:
# 추천에 기여한 정도를 확인
Terminator_2 = title_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_movies_ratings, itemid=Terminator_2)

In [33]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Jurassic Park (1993)', 0.21560856655651567),
 ('Men in Black (1997)', 0.1679387907659144),
 ('Toy Story (1995)', 0.051450299311207565),
 ('Forrest Gump (1994)', -0.013099231340139402),
 ('Titanic (1997)', -0.01711478463153937)]

### 회고
- 내가 좋아하는 영화에 대한 데이터를 데이터프레임에 추가할 때 노드에서 했던 것보다 추가해야되는게 많아서 좀 힘들었음
- 내가 좋아하는 영화와 나의 내적 벡터가 0.55 정도로 1이 아니지만 추천을 받았을 때 생각보다 나쁘지 않게 나옴
- 내가 좋아하는 영화를 추가할 때 영화 장르를 다양하게 넣었는데 만약 장르를 어느 정도 고정해서 추가한다면 결과가 더 잘나오지 않을까 생각함